<a href="https://colab.research.google.com/github/yexf308/AppliedStatistics/blob/main/6_Kernalized_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline 
import numpy.linalg as LA

$\def\m#1{\mathbf{#1}}$
$\def\mm#1{\boldsymbol{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$
$\def\mr#1{\mathrm{#1}}$
$\newenvironment{rmat}{\left[\begin{array}{rrrrrrrrrrrrr}}{\end{array}\right]}$
$\newcommand\brm{\begin{rmat}}$
$\newcommand\erm{\end{rmat}}$
$\newenvironment{cmat}{\left[\begin{array}{ccccccccc}}{\end{array}\right]}$
$\newcommand\bcm{\begin{cmat}}$
$\newcommand\ecm{\end{cmat}}$


# Kernalized Regression

<img src="https://github.com/yexf308/MAT592/blob/main/image/map_to_polar.png?raw=true" width="800" />

<img src="https://github.com/yexf308/MAT592/blob/main/image/map_to_high_d.png?raw=true" width="800" />

Many times data has the nonlinear features. In general, it is easier to classify/regress in high dimensional feature space. However, it is hard to know which feature map will work for given data. So the rule of thumb is to use lots of high-dimensional features and hope our algorithm will automatically pick the right set of features. 

To be more precise, **feature mapping $\mm\phi$**: $\mb{R}^d \rightarrow \mb{R}^p$. It maps the original data into a rich and high-dimensional feature space. ($p\gg d$). 

Example: 

- $d=1$, we can use the following polynomial basis. 
\begin{align}
\mm\phi(x) = \bcm \phi_1(x) \\ \vdots \\ \phi_k(x)\ecm =\bcm x \\ \vdots \\ x^k  \ecm
\end{align}


- $d>1$, we can (randomly) generate vectors $\{\m{u}_j\}_{j=1}^p \subset \mb{R}^d$ and define possible features are listed here
\begin{align}
&\phi_j(\m{x}) = \cos(\m{u}_j^{\top} \m{x}) \\
&\phi_j(\m{x}) = (\m{u}_j^{\top} \m{x})^2 \\ 
&\phi_j(\m{x}) = \frac{1}{1+\exp(\m{u}_j^{\top} \m{x})}
\end{align} 



So feature space cab get really large really quickly. 

**Question:** 

- How many coefficients/parameters are there for degree-$k$ polynomials for $\m{x}=[x_1, x_2, \dots, x_d]\in \mb{R}^d$?

- When $d< N\ll p$, do we need the huge memory to store these features $\{\mm\phi(\m{x}^{(i)})\}_{i=1}^N$ and large run time?? How do we deal with high-dimensional lifts of the data? 

## **A fundamental trick in ML: use kernels.**

**Definition**: A function $K: \mb{R}^d \times \mb{R}^d \rightarrow \mb{R}$ is a **kernel** for a map $\mm\phi$ if $K(\m{x},\m{x}')=\mm\phi(\m{x})\cdot \phi(\m{x}')$ for all $\m{x}, \m{x}'$.

Note $K(\m{x},\m{x}')\in \mb{R}^N\times \mb{R}^N$ matrix. 

**Main idea:** We can represent our **training algorithm** and **prediction rule** as functions of $K(\m{x},\m{x}')$. This we can avoid explicitly computing and storing the high-dimensional features $\{\mm\phi(\m{x}^{(i)})\}_{i=1}^N$. 




## Ridge Regression as Kernels
Note the objective function is 
\begin{align}
\hat{\m{w}}=\arg\min_{\m{w}\in\mb{R}^d}\|\m{y}-\m{X}\m{w}\|_2^2 +\lambda \|\m{w}\|_2^2
\end{align}

Just for exercise, we represent prediction with $\hat{\m{w}}$ using liear kernel defined as $K(\m{x},\m{x}')=\m{x} (\m{x}')^\top$. So $K = \m{X}\m{X}^\top$. 

- Assuming $N< d$, i.e., number of features are more than number of data points. 

- Training: we have the analytical solutions,
\begin{align}
\hat{\m{w}} &= (\m{X}^\top \m{X}+\lambda \m{I}_{d\times d})^{-1}\m{X}^\top \m{y} \\
&= \m{X}^\top (\m{X}\m{X}^\top+\lambda \m{I}_{N\times N})^{-1} \m{y} 
\end{align}
Need proof.
- **Lemma:** $(P^{-1}+B^\top R^{-1}B)^{-1} B^\top R^{-1}= PB^\top(BPB^\top +R)^{-1}$. $B$ may not be a square matrix. 
 
 Proof: It is equivalent with 
 \begin{align}
 (P^{-1}+B^\top R^{-1}B)PB^\top &= B^\top + B^\top R^{-1} B P B^{\top} \\
 & = B^\top R^{-1}(BPB^\top +R)
 \end{align} 
 If $B\in \mb{R}^{m\times n}$, $(BPB^\top +R)\in \mb{R}^{m\times m}$ and $(P^{-1}+B^\top R^{-1}B)\in \mb{R}^{n\times n}$. 


- $P=\m{I}_{d\times d}, B = \m{X}, R= \lambda \m{I}_{N\times N}$, then
\begin{align}
&\text{RHS} = (\m{I}_{d\times d}+\m{X}^\top(\lambda^{-1}\m{I}_{N\times N})\m{X})^{-1}\m{X}^\top (\lambda^{-1}\m{I}_{N\times N}) = (\m{X}^\top \m{X}+\lambda \m{I}_{d\times d})^{-1}\m{X}^\top \\
&\text{LHS} = \m{I}_{d\times d} \m{X}^\top (\m{X}\m{X}^\top + \lambda \m{I}_{N\times N})^{-1}
\end{align}

- Prediction: for a new data $\m{x}\in \mb{R}^d$, the predicted value $y=\m{x}\hat{\m{w}}= \boxed{\m{x}\m{X}^\top} ( \boxed{\m{X}\m{X}^\top}+\lambda \m{I}_{N\times N})^{-1} \m{y} $.

- To make prediction on any future data points, all we need to know is 
\begin{align}
&\c{K}(\m{x}) \triangleq \bcm K(\m{x}, \m{x}^{(1)}) & K(\m{x}, \m{x}^{(2)}) & \dots & K(\m{x}, \m{x}^{(N)})\ecm =\m{x}\m{X}\\
&\m{K}\triangleq \bcm K(\m{x}^{(1)}, \m{x}^{(1)}) &K(\m{x}^{(1)}, \m{x}^{(2)}) & \dots & K(\m{x}^{(1)}, \m{x}^{(N)}) \\ 
K(\m{x}^{(2)}, \m{x}^{(1)}) &K(\m{x}^{(2)}, \m{x}^{(2)}) & \dots & K(\m{x}^{(2)}, \m{x}^{(N)}) \\
\dots & \dots &\dots &\dots \\
K(\m{x}^{(N)}, \m{x}^{(1)}) &K(\m{x}^{(N)}, \m{x}^{(2)}) & \dots & K(\m{x}^{(N)}, \m{x}^{(N)})
\ecm=\m{X}\m{X}^\top  \in \mb{R}^{N\times N}
\end{align}
Even if we run ridge linear regression on feature map $\mm\phi(\m{x})$ we only need to
access the features via kernel $K(\m{x}^{(i)}, \m{x}^{(j)})$ and $K(\m{x}, \m{x}^{(i)})$ and not features $\mm\phi(\m{x})$. 

### Kernel of Polynomial features
Consider polynomial features of degree exactly $k$, for example, $d=2$ and $k=2$, the feature map is 
\begin{align}
\mm\phi(\m{x})= \bcm x_1^2 & x_2^2 & \sqrt{2}x_1x_2\ecm
\end{align}
Then $K(\m{x}, \m{x}')=\mm\phi(\m{x})\mm\phi(\m{x}')^\top=(x_1x_1'+x_2x_2')^2 = (\m{x}^\top \m{x})^2$

For general $d$, 
\begin{align}
\mm\phi(\m{x})= \bcm x_1^2 & \dots & x_d^2 &\sqrt{2}x_1x_2 &\dots & \sqrt{2}x_{d-1}x_d\ecm \in \mb{R}^{d(d+1)/2}
\end{align}
Then $K(\m{x}, \m{x}') = (\m{x}^\top \m{x})^2$ still the same. 

If we want to consider all polynomial features of degree up to degree $k$, for example $k=2, d=2$,
\begin{align}
\mm\phi(\m{x})= \bcm 1& \sqrt{2}x_1& \sqrt{2}x_2& x_1^2 & x_2^2 &\sqrt{2}x_1x_2\ecm
\end{align} 
Then the kernel $K(\m{x}, \m{x}') = (1+\m{x}\m{x}')^2 $.


### Complexity analysis 
- Note that for a data point $\m{x}$, explicitly computing the feature $\mm{\phi}(\m{x})$ takes $p=d^k$ time/memory. However $\c{K}(\m{x})$ will cost $Nd$ in time and $N$ in space.
  - The features are implicit and accessed only via kernels, making it efficient. 

- In terms of training, what are the difference of time and space complexity?   
  - The kernel trick needs $N^2d$ to calculate $K$ and $N^3$ to solve matrix-vector equation. So time complexity is $O(N^3 + N^2d)$. The space complexity is $O(N^2)$.

  - The traditional methods needs time complexity $O(p^3+ p^2N)$ and the space complexity is $O(Np)$. 



# The kernel Trick
Given dataset $\c{D}=\{\m{x}^{(i)},\m{y}^{(i)}\}_{i=1}^N$,  pick a kernel $K: \mb{R}^d\times \mb{R}^d \rightarrow \mb{R}$. 

- For a choice of a loss, use a linear predictor of the form 
$$\hat{\m{w}}= \sum_{i=1}^N \alpha_i (\mm{\phi}(\m{x}^{(i)}))^\top $$
for some $\mm\alpha = [\alpha_1 , \dots, \alpha_N]^\top\in \mb{R}^N$ to be learned. 



- Design an algorithm that finds $\mm\alpha$ while accessing the data only via the kernel matrix $\m{K}$ (i.e., only using $\mm\phi(\m{x}^{(i)})\mm\phi(\m{x}^{(j)})^\top$). 


- Prediction for the new data $\m{x}$ is  $y = \mm\phi(\m{x})\hat{\m{w}} = \sum_{i=1}^N \alpha_i \mm\phi(\m{x})(\mm{\phi}(\m{x}^{(i)}))^\top =  \c{K}(\m{x})\mm\alpha  $.  


### Kernalized ridge regression

Note the objective function is 
\begin{align}
\hat{\m{w}}=\arg\min_{\m{w}\in\mb{R}^d}\sum_{i=1}^N \|y^{(i)}-\mm\phi(\m{x}^{(i)})\m{w}\|_2^2 +\lambda \|\m{w}\|_2^2
\end{align}

- There exists a linear predictor, $\hat{\m{w}}= \sum_{i=1}^N \alpha_i (\mm{\phi}(\m{x}^{(i)}))^\top $ for some $\mm\alpha = [\alpha_1 , \dots, \alpha_N]^\top\in \mb{R}^N$ to be learned. 

- The optimization becomes 
\begin{align}
\hat{\mm\alpha} &= \arg\min_{\mm\alpha \in\mb{R}^N} \sum_{i=1}^N \|y^{(i)}-\sum_{j=1}^N \alpha_j \mm{\phi}(\m{x}^{(i)}) (\mm{\phi}(\m{x}^{(j)}))^\top \|_2^2 + \lambda \sum_{i=1}^N \sum_{j=1}^N \alpha_i\alpha_j  \mm{\phi}(\m{x}^{(i)}) (\mm{\phi}(\m{x}^{(j)}))^\top \\
&=\arg\min_{\mm\alpha \in\mb{R}^N} \|\m{y} - \m{K}\mm\alpha \|_2^2 +\lambda \mm\alpha^\top \m{K}\mm\alpha
\end{align}

- Then $\hat{\mm\alpha}= (\m{K}+\lambda \m{I}_{N\times N})^{-1}\m{y}$.


### The most general case 
- Once we have chosen to use a feature map $\mm\phi(\m{x})\in \mb{R}^p$, and what we want to solve is 
\begin{align}
\hat{\m{w}}= \arg\min_{\m{w}}\sum_{i=1}^N \ell(\mm\phi(\m{x}^{(i)})\m{w}, y^{(i)})
\end{align}
for some convex loss $\ell$.

- Gradient descent update (from initialization $\m{w}^{(0)}=0$ ) that find the optimal solution is
\begin{align}
\m{w}^{(t+1)}=\m{w}^{(t)}-\eta \sum_{i=1}^N\ell'(\mm\phi(\m{x}^{(i)})\m{w}, y^{(i)})(\mm\phi(\m{x}^{(i)}))^\top
\end{align} 

- One crucial observation is that all $\m{w}^{(t)}$  (including the optimal solution $\m{w}^{(\infty)}$) lie on
the subspace spanned by $\{\mm\phi(\m{x}^{(i)}\}_{i=1}^N$ which is an $N$-dimensional subspace
in $\mb{R}^p$. 

- Hence, it is sufficient to look for a solution that is represented as  $\hat{\m{w}}= \sum_{i=1}^N\alpha_i \mm\phi(\m{x}^{(i)})$ to find the optimal solution. 

- Kernel trick finds the optimal solution efficiently, by searching over the model that
can be represented as $\hat{\m{w}}= \sum_{i=1}^N\alpha_i \mm\phi(\m{x}^{(i)})$. 


### Gaussian kernels 
Gaussian (squared exponential) kernels. (a.k.a RBF kernel for Radial Basis Function). 
$$K(\m{x}, \m{x}')= \exp\left(-\frac{\|\m{x}-\m{x}'\|_2^2}{2\sigma^2}\right) $$


---



<img src="https://github.com/yexf308/AppliedStatistics/blob/main/image/rbf1.png?raw=true" width="800" />

Predictor $f(\m{x})= \sum_{i=1}^N \alpha_i K(\m{x}, \m{x}^{(i)})$ is taking weighted sum of $N$ kernel functions
centered at each sample points.

<img src="https://github.com/yexf308/AppliedStatistics/blob/main/image/rbf2.png?raw=true" width="800" />



### When $N$ is very large: use random features.
**Features vs. RBF kernel vs. random features**

If N is very large, allocating an $N\times N$ matrix is tough.
Instead, consider generating random feature maps of the form:
\begin{align}
\mm\phi(\m{x}) = \bcm \sqrt{2}\cos(\m{x}\m{w}_1 + b_1) &\dots&   \sqrt{2}\cos(\m{x}\m{w}_{\tilde p} + b_{\tilde p})\ecm 
\end{align}
where $\m{w}_i\sim \c{N}(0, 2\gamma \m{I}_d)$ and $b_i\sim \text{Uniform}(0, \pi)$ with $\tilde p\ll N\ll p$. 

One can show that (Rahimi, Recht NIPS 2007)
\begin{align}
\mb{E}_{\m{w},b}\left[\frac{1}{p}\mm\phi(\m{x})(\mm\phi(\m{x}'))^\top\right] = \exp(-\gamma \|\m{x}-\m{x}'\|_2^2)
\end{align} 
So this choice of random features approximate the desired RBF kernel with  $\gamma=\frac{1}{2\sigma^2}$. 